In [179]:
import pandas as pd
from openpyxl import load_workbook
import numpy as np
import glob
from pathlib import Path
pd.set_option('display.expand_frame_repr', False)  # Evita que las filas se dividan
pd.set_option('display.max_columns', None)  # Muestra todas las columnas

## Obtencion de las rutas de los archivos implicados

In [180]:
wb = load_workbook('config.xlsx')
ws = wb.active
excelListaPrecios =  ws.cell(row=4, column=2).value
excelListaPreciosLG =  ws.cell(row=5, column=2).value
print(excelListaPrecios)
print(excelListaPreciosLG)

C:\Users\Gonzalo PL\Desktop\Automatizaciones\Automatizacione-EMP1\Codigos Costos\LISTA DE PRECIOS - VENTAS COSTOS 20.10.25- OCTUBRE 2025-4.xlsx
C:\Users\Gonzalo PL\Desktop\Automatizaciones\Automatizacione-EMP1\Codigos Costos\LISTA DE PRECIOS LG - VENTAS COSTOS 30.10.25- OCTUBRE 2025.xlsx


## Apartado para Lista Precios

In [181]:
def extraer_desde_primera_cabecera(ruta_excel, nombre_hoja):
    # 1. Leer hoja sin cabecera
    df = pd.read_excel(ruta_excel, header=None, sheet_name=nombre_hoja)
    df_str = df.astype(str).apply(lambda col: col.str.upper().str.strip())

    # 2. Buscar PRIMERA fila que parezca cabecera
    claves_obligatorias = ["CÓDIGO", "CODIGO", "NOMBRE COMERCIAL"]
    fila_cabecera = None

    for i, fila in df_str.iterrows():
        textos = list(fila.values)
        tiene_codigo = any("CÓDIGO" in t or "CODIGO" in t for t in textos)
        tiene_nombre = any("NOMBRE COMERCIAL" in t for t in textos)
        if tiene_codigo and tiene_nombre:
            fila_cabecera = i
            break

    if fila_cabecera is None:
        raise Exception("❌ No se encontró cabecera con Código y Nombre comercial.")

    # Cabecera oficial (la primera que encontramos)
    columnas_raw = df.iloc[fila_cabecera].tolist()

    registros = []

    # 3. Recorrer filas de datos desde la siguiente fila
    for i in range(fila_cabecera + 1, len(df)):
        fila = df.iloc[i]
        fila_str = df_str.iloc[i]
        celdas_no_nulas = fila.dropna()

        # --- saltar filas totalmente vacías ---
        if celdas_no_nulas.empty:
            continue

        # --- saltar filas que son solo un número (totales) ---
        if len(celdas_no_nulas) == 1 and isinstance(celdas_no_nulas.iloc[0], (int, float)):
            continue

        textos = [t for t in fila_str.values if t != ""]
        texto_unico = textos[0] if len(textos) == 1 else ""

        # --- saltar títulos / categorías / totales ---
        if len(textos) == 1:
            t = texto_unico
            if any(p in t for p in ["LISTA DE PRECIO", "LAVADORA", "SECADORA", "PRENSA", "CABINA", "TOTAL"]):
                continue

        # --- saltar cabeceras repetidas (Ítem, Código, Nombre comercial, etc.) ---
        if any(t.startswith("ÍTEM") or t == "ITEM" for t in textos) and \
           any("CÓDIGO" in t or "CODIGO" in t for t in textos):
            continue

        # 4. Convertir a diccionario usando SIEMPRE la primera cabecera
        dic = dict(zip(columnas_raw, fila.tolist()))
        registros.append(dic)

    # 5. DataFrame final con TODAS las columnas
    df_full = pd.DataFrame(registros)

    # 6. Normalizar nombres de columnas
    df_full.columns = df_full.columns.astype(str).str.upper().str.strip()

    # 7. Mapear a las columnas que te interesan
    #    (por si cambian ligeramente los nombres entre hojas)
    mapa_columnas = {}

    # Código
    for col in ["CÓDIGO", "CODIGO"]:
        if col in df_full.columns:
            mapa_columnas["CÓDIGO"] = col
            break

    # Nombre comercial
    for col in ["NOMBRE COMERCIAL"]:
        if col in df_full.columns:
            mapa_columnas["NOMBRE COMERCIAL"] = col
            break

    # Costo
    for col in ["COSTO COMPASS", "COSTO COMPRA", "COSTO OCTUBRE 2025"]:
        if col in df_full.columns:
            mapa_columnas["COSTO COMPASS"] = col
            break

    # Nos quedamos solo con esas columnas
    df_final = df_full[list(mapa_columnas.values())]
    df_final.columns = list(mapa_columnas.keys())

    # Limpiar filas con código vacío
    df_final = df_final[df_final["CÓDIGO"].notna()]
    df_final = df_final.dropna(how='all', subset=["CÓDIGO", "NOMBRE COMERCIAL", "COSTO COMPASS"])
    return df_final

In [182]:
df_a = pd.ExcelFile(excelListaPrecios)
hojas = df_a.sheet_names
concat_a = []
for h in hojas:
    df = extraer_desde_primera_cabecera(excelListaPrecios,h)
    concat_a.append(df)
sd = pd.concat(concat_a, ignore_index=True)
sd = sd.dropna(how='all')
sd = sd.replace(r'^\s*$', np.nan, regex=True)
sd = sd.dropna(how='all', subset=["CÓDIGO", "NOMBRE COMERCIAL", "COSTO COMPASS"])
sd['COSTO COMPASS'] = sd['COSTO COMPASS'].fillna(0)
resExcel1 = sd.copy()

C:\Users\Gonzalo PL\AppData\Local\Temp\ipykernel_23456\855934692.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sd = sd.replace(r'^\s*$', np.nan, regex=True)


## Apartado para lista de precios Lg

In [183]:
def extraer_desde_primera_cabecera_2(ruta_excel, nombre_hoja):
    # 1) Leer la hoja COMPLETA sin cabecera
    df = pd.read_excel(ruta_excel, header=None, sheet_name=nombre_hoja)

    # Versión en texto (mayúsculas) para poder buscar palabras clave
    df_str = df.astype(str).apply(lambda col: col.str.upper().str.strip())

    # =====================================================
    # 2) Detectar la PRIMERA fila que será la cabecera base
    #    (la que tenga CÓDIGO y DESCRIPCION / NOMBRE COMERCIAL)
    # =====================================================
    fila_cabecera = None

    for i, fila in df_str.iterrows():
        textos = list(fila.values)

        # ¿Hay alguna celda que contenga "CÓDIGO" o "CODIGO"?
        tiene_codigo = any("CÓDIGO" in t or "CODIGO" in t for t in textos)

        # ¿Hay alguna celda que contenga "DESCRIPCION" o "NOMBRE COMERCIAL"?
        tiene_nombre = any(
            "DESCRIPCION" in t or "DESCRIPCIÓN" in t or "NOMBRE COMERCIAL" in t
            for t in textos
        )

        # Primera fila que cumple ambas condiciones → cabecera oficial
        if tiene_codigo and tiene_nombre:
            fila_cabecera = i
            break

    if fila_cabecera is None:
        raise Exception("❌ No se encontró cabecera válida.")

    # Guardamos los nombres originales de la cabecera
    columnas_raw = df.iloc[fila_cabecera].tolist()

    registros = []

    # =====================================================
    # 3) Recorrer TODAS las filas de datos que vienen después
    #    y mapearlas SIEMPRE contra la PRIMERA cabecera
    # =====================================================
    for i in range(fila_cabecera + 1, len(df)):
        fila = df.iloc[i]        # fila con tipos reales (número, fecha, etc.)
        fila_str = df_str.iloc[i]  # la misma fila pero en texto upper()

        # Si la fila está completamente vacía → se ignora
        if fila.dropna().empty:
            continue

        textos = [t for t in fila_str.values if t != ""]

        # Filtrar títulos como "SECADORA ...", "LAVADORA ...", "TOTAL", etc.
        if len(textos) == 1:
            if any(p in textos[0] for p in
                   ["SECADORA", "LAVADORA", "TITAN", "MAX", "TOTAL", "STOCK", "GIANT"]):
                continue

        # Si pasó los filtros, construimos un dict usando SIEMPRE la primera cabecera
        registros.append(dict(zip(columnas_raw, fila.tolist())))

    # Pasamos la lista de diccionarios a DataFrame
    df_full = pd.DataFrame(registros)

    # Normalizamos nombres de columnas: texto, mayúsculas, sin espacios
    df_full.columns = df_full.columns.astype(str).str.upper().str.strip()

    # =====================================================
    # 4) Mapeo flexible de columnas
    #    (por si cambian ligeramente entre hojas)
    # =====================================================
    mapa = {}  # aquí guardamos: nombre_estándar -> nombre_real_en_df_full

    # Columna "CÓDIGO"
    for col in ["CÓDIGO", "CODIGO"]:
        if col in df_full.columns:
            mapa["CÓDIGO"] = col
            break

    # Columna "DESCRIPCION" (acepta varias variantes)
    for col in ["DESCRIPCION", "DESCRIPCIÓN", "NOMBRE COMERCIAL"]:
        if col in df_full.columns:
            mapa["DESCRIPCION"] = col
            break

    # Columna de costo (prioridad: COSTO COMPASS, luego COSTO COMPRA, luego COSTO ACTUAL)
    for col in ["COSTO COMPASS", "COSTO COMPRA", "COSTO ACTUAL"]:
        if col in df_full.columns:
            mapa["COSTO COMPASS"] = col
            break

    # Nos quedamos SOLO con las columnas mapeadas
    df_final = df_full[list(mapa.values())]

    # Renombramos las columnas al nombre estándar: CÓDIGO, DESCRIPCION, COSTO COMPASS
    df_final.columns = list(mapa.keys())

    # =====================================================
    # 5) LIMPIEZA FINAL DE FILAS
    #    - Quitar filas vacías
    #    - Quitar títulos/categorías sin código
    #    - Quitar filas donde DESCRIPCION y COSTO estén vacíos
    # =====================================================

    # Quitar filas donde TODAS las columnas estén NaN
    df_final = df_final.dropna(how="all")

    # Aseguramos que CÓDIGO y DESCRIPCION sean string para poder evaluar
    df_final["CÓDIGO"] = df_final["CÓDIGO"].astype(str).str.strip()
    df_final["DESCRIPCION"] = df_final["DESCRIPCION"].astype(str).str.strip()

    # 5.1 Quitar filas donde el CÓDIGO no es válido (vacío, NaN, etc.)
    mask_codigo_valido = df_final["CÓDIGO"].str.match(r"^[A-Za-z0-9]+$", na=False)
    df_final = df_final[mask_codigo_valido]

    # 5.2 Quitar filas donde DESCRIPCION Y COSTO están vacíos a la vez
    desc_vacia = df_final["DESCRIPCION"].isna() | (df_final["DESCRIPCION"] == "")
    costo_vacio = df_final["COSTO COMPASS"].isna()
    mask_mantener = ~(desc_vacia & costo_vacio)   # mantener todo lo que NO tenga ambas vacías

    df_final = df_final[mask_mantener]

    # 5.3 Resetear índice para que quede limpio
    df_final = df_final.reset_index(drop=True)

    return df_final


In [184]:
resExcel2 = extraer_desde_primera_cabecera_2(excelListaPreciosLG, 'RESUMEN GENERAL LG')
resExcel2 = resExcel2.rename(columns={'CÓDIGO':'CÓDIGO','DESCRIPCION':'NOMBRE COMERCIAL'})

## Uniendo los dos dataframe:

In [185]:
dfExel1 = resExcel1.copy()
dfExcel2 = resExcel2.copy()
dfFinal = pd.concat([dfExel1,dfExcel2])

In [186]:
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.utils import get_column_letter
with pd.ExcelWriter("concatenacion_base_precios_1.xlsx", engine="openpyxl") as writer:
    dfFinal.to_excel(writer, index=False)
    wb = writer.book
    ws = writer.sheets['Sheet1']
    
    for col in ['A',"B"]:
        for cell in ws[col]:
            if cell.value is not None:
                cell.number_format =  "@"

    for cell in ws["C"]:
        if cell.value is not None:
            cell.number_format = "#.##0"
    # Crear la tabla automáticamente
    max_row = ws.max_row
    max_col = ws.max_column
    ref = f"A1:{chr(64+max_col)}{max_row}"  # Rango dinámico según tus datos
    tabla = Table(displayName="TablaPrecios", ref=ref)
    
    # Estilo de tabla
    style = TableStyleInfo(name="TableStyleMedium9",
                           showFirstColumn=False,
                           showLastColumn=False,
                           showRowStripes=True,
                           showColumnStripes=True)
    tabla.tableStyleInfo = style
    ws.add_table(tabla)
    
     # Ajustar ancho de columnas al contenido
    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter  # Obtener letra de la columna
        for cell in col:
            if cell.value:
                # Convertimos a str para calcular longitud
                max_length = max(max_length, len(str(cell.value)))
        # Un pequeño margen extra
        ws.column_dimensions[column].width = max_length + 2

In [187]:
dfFinal.head(100)

,CÓDIGO,NOMBRE COMERCIAL,COSTO COMPASS
0,I21451,LAVADORA INDUSTRIAL,11725.0
1,I22728,LAVADORA INDUSTRIAL,16925.0
2,I22219,LAVADORA INDUSTRIAL,23200.0
3,I22729,LAVADORA INDUSTRIAL,24500.0
4,I22730,LAVADORA INDUSTRIAL,24500.0
...,...,...,...
95,I22749,SINGLE DRYER,2300.0
96,I22227,SECADORA SEMI INDUSTRIAL - APILABLE,4200.0
97,I22228,SECADORA SEMI INDUSTRIAL - APILABLE,4200.0
98,I22637,CALANDRIA DE PLANCHADO,12200.0
